# 🧩 3.2 – Parámetros Variables y Ámbito Anidado (LEGB avanzado)

En este notebook profundizamos en el manejo de **argumentos variables** (`*args`, `**kwargs`) y el sistema de búsqueda de nombres en Python (**LEGB**). Entender estos conceptos es clave para escribir funciones reutilizables y evitar errores sutiles de ámbito.

---
## 🎯 Objetivos
- Dominar `*args` y `**kwargs` para diseñar funciones flexibles.
- Comprender cómo Python busca las variables según el modelo **LEGB**.
- Aplicar funciones anidadas con distintos niveles de ámbito.
- Crear funciones genéricas con control de parámetros dinámicos.

In [ ]:
print('✅ Notebook 3.2 — Parámetros Variables y LEGB avanzado listo para usar.')

---
## 1️⃣ Recordatorio rápido: *args y **kwargs

- `*args` agrupa todos los argumentos posicionales en una tupla.
- `**kwargs` agrupa los argumentos con nombre en un diccionario.

Permiten crear funciones que admitan **número variable de argumentos**.

In [12]:
def mostrar_args(arg1, arg2, *args, edad, nombre, **kwargs):
    print('Posicionales Fijo', arg1, arg2)
    print('Posicionales Dinamico:', args)
    print('ClaveValorFijo', edad, nombre)
    print('Nombrados:', kwargs)

mostrar_args(1, 2, 3, 4, 5, sueldo=333, nombre='Ana', edad=30, profesion='granjero')

Posicionales Fijo 1 2
Posicionales Dinamico: (3, 4, 5)
ClaveValorFijo 30 Ana
Nombrados: {'sueldo': 333, 'profesion': 'granjero'}


✅ Ideal para escribir funciones que puedan recibir muchos parámetros opcionales.

---
## 2️⃣ Ejercicio 1 — Crear una función flexible de impresión

Crea una función `imprimir_formato(*args, **kwargs)` que:
- Imprima los valores posicionales separados por un delimitador definido en `kwargs` (`sep`, por defecto una coma).
- Si `upper=True` está en los kwargs, los imprima en mayúsculas.

💡 *Pista:* usa `'sep' in kwargs` y `kwargs.get('upper', False)`.

In [20]:
# Escribe tu solución aquí...
def imprimir_formato(*args, **kwargs):
    sep = kwargs.get('sep',', ')
    upper = kwargs.get('upper', False)

    valores = [str(a).upper() if upper else str(a) for a in args]
    print(sep.join(valores))
    

imprimir_formato('hola','mundo',3,upper=True, sep='#')

HOLA#MUNDO#3


### ✅ Solución propuesta

In [ ]:
def imprimir_formato(*args, **kwargs):
    sep = kwargs.get('sep', ', ')
    upper = kwargs.get('upper', False)

    valores = [str(a).upper() if upper else str(a) for a in args]
    print(sep.join(valores))

imprimir_formato('manzana', 'pera', 'kiwi')
imprimir_formato('manzana', 'pera', 'kiwi', sep=' | ', upper=True)

✅ Combinar `*args` y `**kwargs` permite definir funciones de propósito general sin necesidad de redefinir sus parámetros.

---
## 3️⃣ Modelo LEGB: Local → Enclosing → Global → Built-in

Python busca los nombres en el siguiente orden:
1. **Local**: dentro de la función actual.
2. **Enclosing**: dentro de funciones externas (si las hay).
3. **Global**: en el módulo.
4. **Built-in**: funciones integradas de Python (`len`, `sum`, etc.).

Veámoslo con un ejemplo anidado.

In [21]:
x = 'global'

def externa():
    x = 'enclosing'
    def interna():
        x = 'local'
        print('Valor dentro de interna:', x)
    interna()
    print('Valor dentro de externa:', x)

externa()
print('Valor global:', x)

Valor dentro de interna: local
Valor dentro de externa: enclosing
Valor global: global


✅ Python **busca de adentro hacia afuera**, y detiene la búsqueda al encontrar el primer nombre que coincide.

---
## 4️⃣ Ejercicio 2 — Uso de `nonlocal`

Modifica el ejemplo anterior para que la función interna **modifique el valor** de la variable `x` definida en la función externa (sin usar `global`).

💡 *Pista:* usa la palabra clave `nonlocal` dentro de la función interna.

In [ ]:
# Escribe tu código aquí...

### ✅ Solución propuesta

In [26]:
x = 'global'

def externa():
    x = 'enclosing'
    def interna():
        nonlocal x
        x = 'modificado'
        print('Dentro de interna:', x)
    interna()
    print('Después de interna:', x)

externa()
print('global', x)

Dentro de interna: modificado
Después de interna: modificado
global global


✅ `nonlocal` permite modificar variables definidas en un **ámbito superior no global**, típico en funciones anidadas.

In [ ]:
def crearContador(rango):
    contador = 0
    callbacks = []

    def get():
        return contador
    def incrementar():
        nonlocal contador 
        for callback in callbacks:
                callback(contador)
        contador = contador + rango 
    def suscribe(funcion):
        nonlocal callbacks
        callbacks.append(funcion)


    return get,incrementar,suscribe


get, incrementar, suscribe = crearContador(10)
# get, incrementar = crearContador(50)



suscribe(lambda x: print('enviar correo', x))

suscribe(lambda x: print('enviar mensaje', x))



incrementar()
incrementar()
print(get())

incrementar()

incrementar()


incrementar()

enviar correo 0
enviar mensaje 0
enviar correo 10
enviar mensaje 10
20
enviar correo 20
enviar mensaje 20
enviar correo 30
enviar mensaje 30
enviar correo 40
enviar mensaje 40


---
## 5️⃣ Ejercicio 3 — Registro de argumentos

Crea una función decoradora `registrar_llamada(func)` que:
- Reciba una función como parámetro.
- Imprima los valores de `*args` y `**kwargs` cada vez que se llama.
- Devuelva el resultado original de la función.

💡 *Pista:* define una función interna que capture los parámetros de llamada y use `*args`, `**kwargs`.

In [ ]:
# 💡 Pista: define una función interna y retorna su ejecución.

### ✅ Solución propuesta

In [ ]:
def registrar_llamada(func):
    def envoltura(*args, **kwargs):
        print(f'📞 Llamando a {func.__name__} con args={args}, kwargs={kwargs}')
        return func(*args, **kwargs)
    return envoltura

@registrar_llamada
def saludar(nombre, saludo='Hola'):
    print(f'{saludo}, {nombre}!')

saludar('Ana')
saludar('Luis', saludo='Buenos días')

✅ Este patrón es la base para los **decoradores**, que veremos en detalle en el notebook 3.3.

---
## 🧠 Resumen del notebook

- `*args` y `**kwargs` permiten manejar argumentos variables.
- El modelo **LEGB** define el orden de búsqueda de variables.
- `nonlocal` sirve para modificar valores en ámbitos intermedios.
- Las funciones anidadas permiten crear comportamientos envolventes.

💡 Próximo paso → **3.3 – Closures y Decoradores: reutilización y envoltura funcional.**